In [1]:
import pyspark
from operator import add, sub
from time import sleep
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.streaming.kafka import KafkaUtils
import json
from pyspark import SparkContext, SparkConf


In [4]:
# conf = SparkConf().set("spark.jars", "/path-to-jar/spark-streaming-kafka-0-8-assembly_2.11-2.2.1.jar")

# sc = SparkContext( conf=conf)
# spark = SparkSession.builder.appName('my_awes').master("spark://bphani.local:7077").config("spark.jars", "/Users/phaneendra/Downloads/spark-sql-kafka-0-10_2.11-2.4.4.jar,/Users/phaneendra/Downloads/kafka-clients-2.4.0.jar").getOrCreate()
spark = SparkSession.builder.appName('my_awes').master("local").config("spark.jars", "/Users/phaneendra/Downloads/spark-sql-kafka-0-10_2.11-2.4.4.jar,/Users/phaneendra/Downloads/kafka-clients-2.4.0.jar").getOrCreate()



In [2]:
agg_match_stats = '/Users/phaneendra/Downloads/agg_match_stats_0.csv'
kill_match_stats = '/Users/phaneendra/Downloads/kill_match_stats_final_0.csv'

In [5]:
adf = spark.read.csv(agg_match_stats, inferSchema=True, header=True)
kdf = spark.read.csv(kill_match_stats, inferSchema=True, header=True)
print(adf.schema)
print(kdf.schema)

StructType(List(StructField(date,StringType,true),StructField(game_size,IntegerType,true),StructField(match_id,StringType,true),StructField(match_mode,StringType,true),StructField(party_size,IntegerType,true),StructField(player_assists,IntegerType,true),StructField(player_dbno,IntegerType,true),StructField(player_dist_ride,DoubleType,true),StructField(player_dist_walk,DoubleType,true),StructField(player_dmg,IntegerType,true),StructField(player_kills,IntegerType,true),StructField(player_name,StringType,true),StructField(player_survive_time,DoubleType,true),StructField(team_id,IntegerType,true),StructField(team_placement,IntegerType,true)))
StructType(List(StructField(killed_by,StringType,true),StructField(killer_name,StringType,true),StructField(killer_placement,DoubleType,true),StructField(killer_position_x,DoubleType,true),StructField(killer_position_y,DoubleType,true),StructField(map,StringType,true),StructField(match_id,StringType,true),StructField(time,IntegerType,true),StructField

In [6]:
print(adf.show())

+--------------------+---------+--------------------+----------+----------+--------------+-----------+------------------+------------------+----------+------------+--------------+-------------------+-------+--------------+
|                date|game_size|            match_id|match_mode|party_size|player_assists|player_dbno|  player_dist_ride|  player_dist_walk|player_dmg|player_kills|   player_name|player_survive_time|team_id|team_placement|
+--------------------+---------+--------------------+----------+----------+--------------+-----------+------------------+------------------+----------+------------+--------------+-------------------+-------+--------------+
|2017-11-26T20:59:...|       37|2U4GBNA0YmnNZYkzj...|       tpp|         2|             0|          1|          2870.724|        1784.84778|       117|           1|      SnuffIes|            1106.32|      4|            18|
|2017-11-26T20:59:...|       37|2U4GBNA0YmnNZYkzj...|       tpp|         2|             0|          1|2938.4

In [ ]:
x.columns

In [ ]:
adf.columns

In [ ]:
print(adf.columns)
print(kdf.columns)
print(str(adf.schema.json()))

In [ ]:
adf.createOrReplaceTempView('adf')
kdf.createOrReplaceTempView('kdf')

In [ ]:
kdf.show()

In [ ]:
sql_query = "SELECT l.*, r.killed_by from adf l JOIN kdf r ON l.`match_id` = r.`match_id` and l.player_name=r.victim_name  "
join_df = spark.sql(sql_query)

In [ ]:
join_df.show()

In [ ]:
join_df.count()

In [ ]:
import pandas as pd

padf = pd.read_csv(agg_match_stats)
pkdf = pd.read_csv(kill_match_stats)

In [ ]:
pjoin_df = pd.merge(padf, pkdf, how='inner', left_on=['match_id', 'player_name'], right_on=['match_id', 'victim_name'])

In [ ]:
len(pjoin_df)

In [ ]:
# val mySchema = StructType(Array(
#  StructField("id", IntegerType),
#  StructField("name", StringType),
#  StructField("year", IntegerType),
#  StructField("rating", DoubleType),
#  StructField("duration", IntegerType)
# ))
movie_schema = StructType().add("id", "integer")\
.add("name", "string").add("year", "integer").add("rating", "double").add("duration", "integer")



df = spark.read.schema(movie_schema).csv('/Users/phaneendra/Downloads/moviedata.csv', inferSchema=True, header=True)

In [ ]:
from pyspark.sql import functions as F

newDf2 = df.withColumn("value", F.to_json(F.struct([F.when(F.col(x)!="  ",df[x]).otherwise(None).alias(x) for x in df.columns])))
newDf2.show(truncate=False)

In [ ]:
newDf2.columns

In [ ]:
#Worked
newDf2.selectExpr("CAST(year AS INTEGER)", "CAST(value AS STRING)") \
  .write \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("topic", "test_kafka_spark") \
  .save()



In [ ]:
#Worked
newDf2.selectExpr("CAST(year AS STRING) AS key", "to_json(struct(*)) AS value") \
  .write \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("topic", "test_kafka_spark_2") \
  .save()

In [ ]:
#Try for SalesRecords

sdf = spark.read.csv('/Users/phaneendra/Downloads/100000SalesRecords.csv', inferSchema=True, header=True)
sdf.columns

In [ ]:
sdf.selectExpr("CAST(Region AS STRING) AS key", "to_json(struct(*)) AS value") \
  .write \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("topic", "test_kafka_spark_3") \
  .save()

In [ ]:
dft1 = spark.readStream.format("kafka").option("kafka.bootstrap.servers", "localhost:9092").option("subscribe", "topic-one").option("startingOffsets", "latest").load()



In [ ]:
dft1.writeStream.format("kafka").option("topic", "topic-one").option("kafka.bootstrap.servers", "localhost:9092").option("checkpointLocation","/tmp/").start()




In [ ]:
prob_schema = StructType().add("CUST_ID", "string").add("OUTBUFFER__1", "string")
# Read CSV files from set path
dfCSV = spark.readStream.option("sep", ",").option("header", "true").schema(prob_schema).csv("/Users/phaneendra/Downloads/prob*.csv")


In [ ]:
dfCSV.createOrReplaceTempView("prob")
total = spark.sql("select count(CUST_ID) from prob")


In [ ]:
dfCSV \
.write  \
.format("kafka") \
.option("kafka.batch.size", 5000) \
.option("kafka.bootstrap.servers", "localhost:9092") \
.option("kafka.request.timeout.ms", 120000) \
.option("topic", "raw") \
.option("checkpointLocation", "/mnt/telemetry/cp.txt") \
.save()

In [ ]:
query = total.writeStream.outputMode("complete").format("console").start()
query.awaitTermination()

In [ ]:
query.close()

In [ ]:
dfCSV.selectExpr("CAST(CUST_ID AS STRING) AS key", "to_json(struct(*)) AS value").
  writeStream
  .format("kafka")
  .option("topic", "topic-one")
  .option("kafka.bootstrap.servers", "localhost:9092")
  .start()